In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import re

### Get list of snp ID by fabian format 
file = "/mnt/project/exonhancer/ZENODO_REPO/gnomAD_v3/2_snps_dictionaries/SNP_annot_allExons_fabian_gnomadV3_all.tsv"
dic_fabian_id = {}

with open(file) as f:
    for line in f:
        dic_fabian_id[line.strip().split('\t')[0]] = line.strip().split('\t')[1]

### Get list of sequences with a stong snp and the type of the strongest one 
file = "/home/mouren/Data/valid_exp_starr/list_raw_v3/K562_Dic_SnpStrongType.tsv"
dic_snp = {}

with open(file) as f:
    for line in f:
        dic_snp[line.strip().split('\t')[0]] = line.strip().split('\t')[1]

In [ ]:
file = Path("K562_REF_sequences_raw.tsv")

list_one_snp = []
content = []
with open(file) as f:
    for line in f:
        start = int((line.strip().split()[2]))
        end = int((line.strip().split()[3]))
        name = line.strip().split()[0]

        list_snp = {}
        list_snp_weak = {}

        ### First get all the snp of corresonding effect 
        if name not in dic_snp: #not a sequence to mut 
            continue
   
        df = pd.read_csv("/mnt/project/exonhancer/ZENODO_REPO/gnomAD_v3/1_compute_fabian/pipeline_exons/fabian_result/"+name+".vcf_fabian_bilan.tsv", index_col=0, sep='\t')
        for column_name in df.columns:  
            coord_fabian_tmp = (column_name.split(":")[1]).split(">")[0]
            coord_fabian = [int(x) for x in re.findall(r'\d+', coord_fabian_tmp)] 
            ref = [x for x in re.findall(r'[a-zA-Z]+', coord_fabian_tmp)][0]
            alt = (column_name.split(">")[1]).split(".")[0]

            if len(ref)>1 or len(alt)>1: #remove indel
                continue

            if int(coord_fabian[0]) > start and int(coord_fabian[0]) <= end: #Mutation can be in sequence #> start because start is not included
                try: #If not in the dictionnary it means it didn't pass the gnomAD filter 
                    if column_name.split('.')[0] in dic_fabian_id: #Mutation is in dic 
                        df[column_name] = df[column_name].astype(float)
                        if dic_snp[name] == "LOSS":
                            if any(df[column_name] < -0.33) : 
                                list_snp[column_name] = df[column_name].min()
                            elif any(df[column_name] < 0) : 
                                list_snp_weak[column_name] = df[column_name].min()
                        else:
                            if any(df[column_name] > 0.33) : 
                                list_snp[column_name] = df[column_name].max()
                            elif any(df[column_name] > 0) : 
                                list_snp_weak[column_name] = df[column_name].min()
                except KeyError:
                    continue
            
        list_snp_ordered = dict(sorted(list_snp.items(), key=lambda item: abs(item[1]), reverse=True))
        list_snp_weak_ordered = dict(sorted(list_snp_weak.items(), key=lambda item: abs(item[1]), reverse=True))

        ### Mutate sequence
        position_affected = []

        if line.strip().split()[4] == "+":
            seq = line.strip().split()[5]

            for key,val in list_snp_ordered.items():
                coord_fabian_tmp = (key.split(":")[1]).split(">")[0]
                coord_fabian = [int(x) for x in re.findall(r'\d+', coord_fabian_tmp)] 
                ref = [x for x in re.findall(r'[a-zA-Z]+', coord_fabian_tmp)][0]
                alt = (key.split(">")[1]).split(".")[0]

                #Check if the mutation is not next to another one 
                skip = False 
                for position in position_affected:
                    if int(coord_fabian[0])+1 == position or int(coord_fabian[0])-1 == position or int(coord_fabian[0]) == position:
                        skip = True
                        break
                if skip:
                    continue

                cpt = start+1 #+1 because in UCSC coordinates the start is not included
                
                for base in range(len(seq)):
                    if cpt != int(coord_fabian[0]):
                        cpt += 1
                        continue
                    else:
                        if ref != seq[base]:
                            print(print(key), ref, alt)
                            print(seq[base])
                            break
                        else:
                            seq = seq[:base] + alt.lower() + seq[base+1:]
                            position_affected.append(int(coord_fabian[0]))
                            break
            
            if len(position_affected) <= 2: #We want at least 3 SNP in the sequence
                for key,val in list_snp_weak_ordered.items():
                    coord_fabian_tmp = (key.split(":")[1]).split(">")[0]
                    coord_fabian = [int(x) for x in re.findall(r'\d+', coord_fabian_tmp)] 
                    ref = [x for x in re.findall(r'[a-zA-Z]+', coord_fabian_tmp)][0]
                    alt = (key.split(">")[1]).split(".")[0]

                    #Check if the mutation is not next to another one
                    skip = False 
                    for position in position_affected:
                        if int(coord_fabian[0])+1 == position or int(coord_fabian[0])-1 == position or int(coord_fabian[0]) == position:
                            skip = True
                            break
                    if skip:
                        continue

                    cpt = start+1 #+1 because in UCSC coordinates the start is not included
                    
                    for base in range(len(seq)):
                        if cpt != int(coord_fabian[0]):
                            cpt += 1
                            continue
                        else:
                            if ref != seq[base]:
                                print(print(key), ref, alt)
                                print(seq[base])
                                break
                            else:
                                seq = seq[:base] + alt.lower() + seq[base+1:]
                                position_affected.append(int(coord_fabian[0]))
                                break

        else: #Negative strand so sequence is RC
            seq = (line.strip().split()[5])[::-1]
            nuc = {"A":"T","T":"A","C":"G","G":"C"}

            for key,val in list_snp_ordered.items():
                coord_fabian_tmp = (key.split(":")[1]).split(">")[0]
                coord_fabian = [int(x) for x in re.findall(r'\d+', coord_fabian_tmp)] 
                ref = [x for x in re.findall(r'[a-zA-Z]+', coord_fabian_tmp)][0]
                alt = (key.split(">")[1]).split(".")[0]
                ref2 = nuc[ref]
                alt2 = nuc[alt]

                #Check if the mutation is not next to another one 
                skip = False 
                for position in position_affected:
                    if int(coord_fabian[0])+1 == position or int(coord_fabian[0])-1 == position or int(coord_fabian[0]) == position:
                        skip = True
                        break
                if skip:
                    continue

                cpt = start+1 #+1 because in UCSC coordinates the start is not included
                
                for base in range(len(seq)):
                    if cpt != int(coord_fabian[0]):
                        cpt += 1
                        continue
                    else:
                        if ref2 != seq[base]:
                            print(print(key), ref, alt)
                            print(seq[base])
                            break
                        else:
                            seq = seq[:base] + alt2.lower() + seq[base+1:]
                            position_affected.append(int(coord_fabian[0]))
                            break
            
            if len(position_affected) <= 2: #We want at least 3 SNP in the sequence
                for key,val in list_snp_weak_ordered.items():
                    coord_fabian_tmp = (key.split(":")[1]).split(">")[0]
                    coord_fabian = [int(x) for x in re.findall(r'\d+', coord_fabian_tmp)] 
                    ref = [x for x in re.findall(r'[a-zA-Z]+', coord_fabian_tmp)][0]
                    alt = (key.split(">")[1]).split(".")[0]
                    ref2 = nuc[ref]
                    alt2 = nuc[alt]

                    #Check if the mutation is not next to another one
                    skip = False 
                    for position in position_affected:
                        if int(coord_fabian[0])+1 == position or int(coord_fabian[0])-1 == position or int(coord_fabian[0]) == position:
                            skip = True
                            break
                    if skip:
                        continue

                    cpt = start+1 #+1 because in UCSC coordinates the start is not included
                    
                    for base in range(len(seq)):
                        if cpt != int(coord_fabian[0]):
                            cpt += 1
                            continue
                        else:
                            if ref2 != seq[base]:
                                print(print(key), ref, alt)
                                print(seq[base])
                                break
                            else:
                                seq = seq[:base] + alt2.lower() + seq[base+1:]
                                position_affected.append(int(coord_fabian[0]))
                                break
            seq = seq[::-1]

        if dic_snp[name] == "LOSS":
            name2 = str("_".join(name.split('_')[:3]))+"_L_EEMULTISNP" 
        else:
            name2 = str("_".join(name.split('_')[:3]))+"_G_EEMULTISNP" 

        entry = ["EEMULTISNP",name2,line.strip().split()[1],start,end,line.strip().split()[4],seq]
        content.append(entry)

df = pd.DataFrame(content)
df.to_csv("K562_MultiMUT_sequences.tsv",sep="\t",header=False,index=False)